<a href="https://colab.research.google.com/github/Sinandah/Sinandah/blob/main/customer_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Import necessary libraries
import pandas as pd
from urllib.request import urlretrieve

# URL for the Online Retail Data
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"

# Define the file name for saving
file_name = "Online_Retail.xlsx"

# Download the dataset
urlretrieve(url, file_name)

# Load the dataset into a Pandas DataFrame
df = pd.read_excel(file_name
)

# Display the first few rows of the dataset
df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [6]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_excel("Online_Retail.xlsx")

# Display the first few rows of the dataset
print("First few rows of the dataset:")
print(df.head())

# Summary statistics
print("\nSummary statistics of numeric columns:")
print(df.describe())

# Check for missing values
print("\nMissing values in the dataset:")
print(df.isnull().sum())

# Explore unique values in categorical columns
print("\nUnique values in categorical columns:")
for column in df.select_dtypes(include=['object']).columns:
    print(f"{column}: {df[column].nunique()} unique values")

# Visualizations

# Distribution of purchase amounts
plt.figure(figsize=(10, 6))
sns.histplot(df['AmountSpent'], bins=50, kde=True)
plt.title('Distribution of Purchase Amounts')
plt.xlabel('Amount Spent')
plt.ylabel('Frequency')
plt.show()

# Monthly order count
df['InvoiceMonth'] = df['InvoiceDate'].dt.to_period('M')
monthly_order_count = df.groupby('InvoiceMonth')['InvoiceNo'].nunique()

plt.figure(figsize=(12, 6))
monthly_order_count.plot(marker='o')
plt.title('Monthly Order Count')
plt.xlabel('Month')
plt.ylabel('Number of Orders')
plt.show()

# Top products by quantity sold
top_products = df.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(12, 6))
top_products.plot(kind='barh', color='skyblue')
plt.title('Top 10 Products by Quantity Sold')
plt.xlabel('Quantity Sold')
plt.ylabel('Product Description')
plt.show()


First few rows of the dataset:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  

Summary statistics of numeric columns:
            Quantity      UnitPrice     CustomerID
count  541909.000000  541909.000000  406829.000000
mean        9.55

KeyError: ignored

<Figure size 1000x600 with 0 Axes>

In [4]:
# Import necessary libraries
import pandas as pd

# Load the dataset
df = pd.read_excel("Online_Retail.xlsx")

# Display the number of missing values in each column
print("Missing values before data cleaning:")
print(df.isnull().sum())

# Data Cleaning

# Drop rows with missing values in critical columns
df = df.dropna(subset=['CustomerID', 'InvoiceNo', 'Description'])

# Convert CustomerID to integer type
df['CustomerID'] = df['CustomerID'].astype(int)

# Convert InvoiceDate to datetime type
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Drop rows with negative or zero quantity
df = df[df['Quantity'] > 0]

# Drop rows with negative or zero unit price
df = df[df['UnitPrice'] > 0]

# Data Preprocessing

# Create a new column for total purchase amount
df['AmountSpent'] = df['Quantity'] * df['UnitPrice']

# Display the number of missing values after data cleaning
print("\nMissing values after data cleaning:")
print(df.isnull().sum())

# Save the cleaned and preprocessed dataset
df.to_csv("cleaned_online_retail.csv", index=False)


Missing values before data cleaning:
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


<ipython-input-4-4d8b6e414274>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CustomerID'] = df['CustomerID'].astype(int)



Missing values after data cleaning:
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
AmountSpent    0
dtype: int64


In [13]:
# Import necessary libraries
import pandas as pd

# Load the cleaned dataset
df = pd.read_csv("cleaned_online_retail.csv")

# Feature Engineering

# Extract month and year from InvoiceDate
df['InvoiceMonth'] = df['InvoiceDate'].dt.month
df['InvoiceYear'] = df['InvoiceDate'].dt.year

# Calculate the average purchase amount per transaction
average_purchase_amount = df.groupby('InvoiceNo')['AmountSpent'].mean()
df['AveragePurchaseAmount'] = df['InvoiceNo'].map(average_purchase_amount)

# Calculate the total number of unique products purchased by each customer
unique_products_per_customer = df.groupby('CustomerID')['Description'].nunique()
df['UniqueProductsPerCustomer'] = df['CustomerID'].map(unique_products_per_customer)

# Calculate the total number of transactions per customer
total_transactions_per_customer = df.groupby('CustomerID')['InvoiceNo'].nunique()
df['TotalTransactions'] = df['CustomerID'].map(total_transactions_per_customer)

# Display the first few rows of the dataset after feature engineering
print(df.head())

# Save the dataset with engineered features
df.to_csv("engineered_online_retail.csv", index=False)


AttributeError: ignored

In [12]:
# Import necessary libraries
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Load the engineered dataset
df = pd.read_csv("engineered_online_retail.csv")

# Select features for segmentation
features = ['InvoiceMonth', 'AveragePurchaseAmount', 'UniqueProductsPerCustomer', 'TotalTransactions']
X = df[features]

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Determine the optimal number of clusters using the Elbow Method
inertia = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

# Plot the Elbow Method
plt.figure(figsize=(8, 5))
plt.plot(range(1, 11), inertia, marker='o')
plt.title('Elbow Method for Optimal Number of Clusters')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia (Within-Cluster Sum of Squares)')
plt.show()

# Based on the Elbow Method, choose the optimal number of clusters (e.g., 3)
optimal_clusters = 3

# Apply k-means clustering with the optimal number of clusters
kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
df['Cluster'] = kmeans.fit_predict(X_scaled)

# Display the distribution of customers across clusters
print("Distribution of customers across clusters:")
print(df['Cluster'].value_counts())

# Save the dataset with cluster assignments
df.to_csv("segmented_online_retail.csv", index=False)


FileNotFoundError: ignored

In [14]:
from sklearn.metrics import silhouette_score, davies_bouldin_score

# Load the segmented dataset
df = pd.read_csv("segmented_online_retail.csv")

# Select features for evaluation
X_eval = df[features]

# Standardize the evaluation features
X_eval_scaled = scaler.transform(X_eval)

# Calculate silhouette score
silhouette_avg = silhouette_score(X_eval_scaled, df['Cluster'])
print(f"Silhouette Score: {silhouette_avg}")

# Calculate Davies-Bouldin index
davies_bouldin_idx = davies_bouldin_score(X_eval_scaled, df['Cluster'])
print(f"Davies-Bouldin Index: {davies_bouldin_idx}")


FileNotFoundError: ignored

In [10]:
import matplotlib.pyplot as plt
import seaborn as sns

# Load the segmented dataset
df = pd.read_csv("segmented_online_retail.csv")

# Scatter plot for visualization (example with two features)
plt.figure(figsize=(10, 6))
sns.scatterplot(x='AveragePurchaseAmount', y='TotalTransactions', hue='Cluster', data=df, palette='viridis', s=50)
plt.title('Customer Segmentation')
plt.xlabel('Average Purchase Amount')
plt.ylabel('Total Transactions')
plt.legend(title='Cluster')
plt.show()


FileNotFoundError: ignored

In [ ]:
# Pair plot for visualization of multiple features
features_to_plot = ['InvoiceMonth', 'AveragePurchaseAmount', 'UniqueProductsPerCustomer', 'TotalTransactions']

plt.figure(figsize=(12, 8))
sns.pairplot(df, vars=features_to_plot, hue='Cluster', palette='viridis', markers='o')
plt.suptitle('Pair Plot of Customer Segmentation', y=1.02)
plt.show()
